# Neural Machine Translation

**Reference:** Sutskever, Ilya, Oriol Vinyals, and Quoc V. Le. "Sequence to sequence learning with neural networks." In Advances in neural information processing systems, pp. 3104-3112. 2014. ([Paper](https://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks), [Sample code](https://github.com/tensorflow/nmt))

In [1]:
using Knet, Test, Base.Iterators, IterTools, Random # , LinearAlgebra, StatsBase
using AutoGrad: @gcheck  # to check gradients, use with Float64
Knet.atype() = KnetArray{Float32}  # determines what Knet.param() uses.
macro size(z, s); esc(:(@assert (size($z) == $s) string(summary($z),!=,$s))); end # for debugging

┌ Info: Recompiling stale cache file /home/ec2-user/.julia/compiled/v1.2/Knet/f4vSz.ji for Knet [1902f260-5fb4-5aff-8c31-6271790ab950]
└ @ Base loading.jl:1240


@size (macro with 1 method)

## Part -1. Types from the last project

Please copy the following types and related functions from the last project: `Vocab`,
`TextReader`, `Embed`, `Linear`, `mask!`.

In [2]:
struct Vocab
    w2i::Dict{String,Int}
    i2w::Vector{String}
    unk::Int
    eos::Int
    tokenizer
end

function Vocab(file::String; tokenizer=split, vocabsize=Inf, mincount=1, unk="<unk>", eos="<s>")
    word_count = Dict{String,Int}()
    w2i = Dict{String,Int}()
    i2w = Vector{String}()
    int_eos = get!(w2i, eos, 1+length(w2i))
    int_unk = get!(w2i, unk, 1+length(w2i))
    for line in eachline(file)
        line = tokenizer(line)
        for word in line
            if haskey(word_count, word)
                word_count[word] += 1
            else
                word_count[word] = 1
            end
        end
    end
    word_count = collect(word_count)
    sort!(word_count, rev=true, by=x->x[2])
    # constructing w2i
    for pair in word_count
        if pair[2] >= mincount
            get!(w2i, pair[1], 1+length(w2i))
            if length(w2i) >= vocabsize
                break
            end
        end
    end
    w2i_array = collect(w2i)
    sort!(w2i_array, by=x->x[2])
    for pair in w2i_array
        push!(i2w, pair[1])
    end
    return Vocab(w2i, i2w, int_unk, int_eos, tokenizer)
end

Vocab

In [3]:
struct TextReader
    file::String
    vocab::Vocab
end

function Base.iterate(r::TextReader, s=nothing)
    s == nothing && (s = open(r.file))
    if eof(s)
        close(s)
        return nothing
    end
    line = readline(s)
    line = r.vocab.tokenizer(line)
    line_inds = Int[]
    for word in line
        push!(line_inds, get(r.vocab.w2i, word, r.vocab.unk))
    end
    return line_inds, s
end

Base.IteratorSize(::Type{TextReader}) = Base.SizeUnknown()
Base.IteratorEltype(::Type{TextReader}) = Base.HasEltype()
Base.eltype(::Type{TextReader}) = Vector{Int}

In [4]:
struct Embed; w; end

function Embed(vocabsize::Int, embedsize::Int)
    return Embed(param(embedsize, vocabsize, atype = Knet.atype()))
end

function (l::Embed)(x)
    l.w[:,x]
end

In [5]:
struct Linear; w; b; end

function Linear(inputsize::Int, outputsize::Int)
    return Linear(param(outputsize, inputsize, atype = Knet.atype()), param0(outputsize, atype = Knet.atype()))
end

function (l::Linear)(x)
    l.w * x .+ l.b
end

In [6]:
function mask!(a,pad)
    num_rows, num_cols = size(a)
    for row in 1:num_rows
        for column in num_cols:-1:1
            if a[row, column] != pad || a[row, column-1] != pad
                break
            else
                a[row, column] = 0
            end
        end
    end
    return a
end

mask! (generic function with 1 method)

## Part 0. Load data

We will use the Turkish-English pair from the [TED Talks Dataset](https://github.com/neulab/word-embeddings-for-nmt) for our experiments.

In [7]:
datadir = "datasets/tr_to_en"

if !isdir(datadir)
    download("http://www.phontron.com/data/qi18naacl-dataset.tar.gz", "qi18naacl-dataset.tar.gz")
    run(`tar xzf qi18naacl-dataset.tar.gz`)
end

if !isdefined(Main, :tr_vocab)
    tr_vocab = Vocab("$datadir/tr.train", mincount=5)
    en_vocab = Vocab("$datadir/en.train", mincount=5)
    tr_train = TextReader("$datadir/tr.train", tr_vocab)
    en_train = TextReader("$datadir/en.train", en_vocab)
    tr_dev = TextReader("$datadir/tr.dev", tr_vocab)
    en_dev = TextReader("$datadir/en.dev", en_vocab)
    tr_test = TextReader("$datadir/tr.test", tr_vocab)
    en_test = TextReader("$datadir/en.test", en_vocab)
    @info "Testing data"
    @test length(tr_vocab.i2w) == 38126
    @test length(first(tr_test)) == 16
    @test length(collect(tr_test)) == 5029
end

┌ Info: Testing data
└ @ Main In[7]:17


Test Passed

## Part 1. Minibatching

For minibatching we are going to design a new iterator: `MTData`. This iterator is built
on top of two TextReaders `src` and `tgt` that produce parallel sentences for source and
target languages.

In [8]:
struct MTData
    src::TextReader        # reader for source language data
    tgt::TextReader        # reader for target language data
    batchsize::Int         # desired batch size
    maxlength::Int         # skip if source sentence above maxlength
    batchmajor::Bool       # batch dims (B,T) if batchmajor=false (default) or (T,B) if true.
    bucketwidth::Int       # batch sentences with length within bucketwidth of each other
    buckets::Vector        # sentences collected in separate arrays called buckets for each length range
    batchmaker::Function   # function that turns a bucket into a batch.
end

function MTData(src::TextReader, tgt::TextReader; batchmaker = arraybatch, batchsize = 128, maxlength = typemax(Int),
                batchmajor = false, bucketwidth = 10, numbuckets = min(128, maxlength ÷ bucketwidth))
    buckets = [ [] for i in 1:numbuckets ] # buckets[i] is an array of sentence pairs with similar length
    MTData(src, tgt, batchsize, maxlength, batchmajor, bucketwidth, buckets, batchmaker)
end

Base.IteratorSize(::Type{MTData}) = Base.SizeUnknown()
Base.IteratorEltype(::Type{MTData}) = Base.HasEltype()
Base.eltype(::Type{MTData}) = NTuple{2}

### iterate(::MTData)

Define the `iterate` function for the `MTData` iterator. `iterate` should return a
`(batch, state)` pair or `nothing` if there are no more batches.  The `batch` is a
`(x::Matrix{Int},y::Matrix{Int})` pair where `x` is a `(batchsize,srclength)` batch of
source language sentences and `y` is a `(batchsize,tgtlength)` batch of parallel target
language translations. The `state` is a pair of `(src_state,tgt_state)` which can be used
to iterate `d.src` and `d.tgt` to get more sentences.  `iterate(d)` without a second
argument should initialize `d` by emptying its buckets and calling `iterate` on the inner
iterators `d.src` and `d.tgt` without a state. Please review the documentation on
iterators from the last project.

To keep similar length sentences together `MTData` uses arrays of similar length sentence
pairs called buckets.  Specifically, the `(src_sentence,tgt_sentence)` pairs coming from
`src` and `tgt` are pushed into `d.buckets[i]` when the length of the source sentence is
in the range `((i-1)*d.bucketwidth+1):(i*d.bucketwidth)`. When one of the buckets reaches
`d.batchsize` `d.batchmaker` is called with the full bucket producing a 2-D batch, the
bucket is emptied and the batch is returned. If `src` and `tgt` are exhausted the
remaining partially full buckets are turned into batches and returned in any order. If the
source sentence length is larger than `length(d.buckets)*d.bucketwidth`, the last bucket
is used.

Sentences above a certain length can be skipped using the `d.maxlength` field, and
transposed `x,y` arrays can be produced using the `d.batchmajor` field.

In [30]:
function Base.iterate(d::MTData, state=nothing)
    # Your code here
    if state == nothing
        for i in 1:length(d.buckets)
            d.buckets[i] = []
        end
    end
      
    while true
        if state == nothing
            s_data,s_state = iterate(d.src)
            t_data,t_state = iterate(d.tgt)
        elseif state == "eof"
             #if there are no more half filled batches, return nothing
            for i in 1:length(d.buckets)
                if length(d.buckets[i]) > 0
                    bucket = d.buckets[i]
                    d.buckets[i] = []
                    return (d.batchmaker(d,bucket),"eof")
                end
            end
            
            #no more batches
            return nothing
        else
            src = iterate(d.src,state[1]) 
            tgt = iterate(d.tgt,state[2])
            
            #TextReader returns a "nothing" value when it finishes, so i need to turn it into a Tuple for 
            #my code to run as intended
            if src == nothing; src = (nothing,nothing);end
            if tgt == nothing; tgt = (nothing,nothing);end

            s_data,s_state = src
            t_data,t_state = tgt
        end
        
        
        #if we reached the end of the src or tgt files, return half filled buckets
        if (s_state == nothing) | (t_state ==  nothing)
            state = "eof"
            continue
        end
        
        #if source sentence above d.maxlength
        if length(s_data) > d.maxlength
            state = (s_state,t_state)
            continue
        #if src sentence larger than a condition specified by Deniz
        elseif length(s_data) >=  length(d.buckets)*d.bucketwidth
            
            push!(d.buckets[length(d.buckets)],(s_data,t_data))
            if length(d.buckets[length(d.buckets)]) == d.batchsize
                bucket = d.buckets[length(d.buckets)]
                d.buckets[length(d.buckets)] = []
                return (d.batchmaker(d,bucket),(s_state,t_state))
            end
        #of if src sentence in range
            
        else
            
            for i in 1:length(d.buckets)
                if length(s_data) in ((i-1)*d.bucketwidth+1):(i*d.bucketwidth)
                    push!(d.buckets[i],(s_data,t_data))
                    if length(d.buckets[i]) == d.batchsize
                        bucket = d.buckets[i]
                        d.buckets[i] = []
                        return (d.batchmaker(d,bucket),(s_state,t_state))
                    end
                end
            end
            
        end
        state = (s_state,t_state)
    end
end

### arraybatch

Define `arraybatch(d, bucket)` to be used as the default `d.batchmaker`. `arraybatch`
takes an `MTData` object and an array of sentence pairs `bucket` and returns a
`(x::Matrix{Int},y::Matrix{Int})` pair where `x` is a `(batchsize,srclength)` batch of
source language sentences and `y` is a `(batchsize,tgtlength)` batch of parallel target
language translations. Note that the sentences in the bucket do not have any `eos` tokens
and they may have different lengths. `arraybatch` should copy the source sentences into
`x` padding shorter ones on the left with `eos` tokens. It should copy the target
sentences into `y` with an `eos` token in the beginning and end of each sentence and
shorter sentences padded on the right with extra `eos` tokens.

In [31]:
function arraybatch(d::MTData, bucket)
    # Your code here
     # Your code here
    x = [] 
    y = []
    
    srclength = max([length(pair[1]) for pair in bucket]...)
    tgtlength = max([length(pair[2]) for pair in bucket]...)
    
    for pair in bucket
        src_sentence, tgt_sentence = pair
        src_sen = []
        tgt_sen = []
                
        tgt_eos = d.tgt.vocab.eos
        push!(tgt_sen, tgt_eos)
        for w in tgt_sentence
            push!(tgt_sen, w) 
        end
        while length(tgt_sen) != (tgtlength + 2)
            push!(tgt_sen, tgt_eos)
        end
        
        src_eos = d.src.vocab.eos
        eos_num = srclength - length(src_sentence)
        i = 0
        while i!= eos_num
            push!(src_sen, src_eos)
            i += 1
        end
        for w in src_sentence
            push!(src_sen, w) 
        end        
        push!(x, src_sen)
        push!(y, tgt_sen)
    end
    
    return Matrix(hcat(x...)'), Matrix(hcat(y...)')
end

arraybatch (generic function with 1 method)

In [32]:
@info "Testing MTData"
dtrn = MTData(tr_train, en_train)
ddev = MTData(tr_dev, en_dev)
dtst = MTData(tr_test, en_test)

x,y = first(dtst)
@test length(collect(dtst)) == 48
@test size.((x,y)) == ((128,10),(128,24))
@test x[1,1] == tr_vocab.eos
@test x[1,end] != tr_vocab.eos
@test y[1,1] == en_vocab.eos
@test y[1,2] != en_vocab.eos
@test y[1,end] == en_vocab.eos

┌ Info: Testing MTData
└ @ Main In[32]:1


Test Passed

## Part 2. Sequence to sequence model without attention

In this part we will define a simple sequence to sequence encoder-decoder model for
machine translation.

In [12]:
struct S2S_v1
    srcembed::Embed     # source language embedding
    encoder::RNN        # encoder RNN (can be bidirectional)
    tgtembed::Embed     # target language embedding
    decoder::RNN        # decoder RNN
    projection::Linear  # converts decoder output to vocab scores
    dropout::Real       # dropout probability to prevent overfitting
    srcvocab::Vocab     # source language vocabulary
    tgtvocab::Vocab     # target language vocabulary
end

### S2S_v1 constructor

Define the S2S_v1 constructor using your predefined layer types (Embed, Linear), and the
Knet RNN type. Please review the RNN documentation using `@doc RNN`, paying attention to
the following options in particular: `numLayers`, `bidirectional`, `dropout`, `dataType`,
`usegpu`. The last two are important if you experiment with array types other than the
default `KnetArray{Float32}`: make sure the RNNs use the same array type as the other
layers. Note that if the encoder is bidirectional, its `numLayers` should be half of the
decoder so that their hidden states match in size.

In [13]:
function S2S_v1(hidden::Int,         # hidden size for both the encoder and decoder RNN
                srcembsz::Int,       # embedding size for source language
                tgtembsz::Int,       # embedding size for target language
                srcvocab::Vocab,     # vocabulary for source language
                tgtvocab::Vocab;     # vocabulary for target language
                layers=1,            # number of layers
                bidirectional=false, # whether encoder RNN is bidirectional
                dropout=0)           # dropout probability
    
    srcembed = Embed(length(srcvocab.i2w), srcembsz)
    encoder = RNN(srcembsz, hidden, numLayers=(bidirectional ? Integer(layers/2) : layers), bidirectional=bidirectional, dropout=dropout, dataType=Float32, usegpu=(gpu()>=0))
    tgtembed = Embed(length(tgtvocab.i2w), tgtembsz)
    decoder = RNN(tgtembsz, hidden, numLayers=layers, dropout=dropout, dataType=Float32, usegpu=(gpu()>=0))
    projection = Linear(hidden, length(tgtvocab.i2w))
    
    return S2S_v1(srcembed, encoder, tgtembed, decoder, projection, dropout, srcvocab, tgtvocab)
end

S2S_v1

### S2S_v1 loss function

Define the S2S_v1 loss function that takes `src`, a source language minibatch, and `tgt`,
a target language minibatch and returns either a `(total_loss, num_words)` pair if
`average=false`, or `(total_loss/num_words)` average if `average=true`.

Assume that `src` and `tgt` are integer arrays of size `(B,Tx)` and `(B,Ty)` respectively,
where `B` is the batch size, `Tx` is the length of the longest source sequence, `Ty` is
the length of the longest target sequence. The `src` sequences only contain words, the
`tgt` sequences surround the words with `eos` tokens at the start and end. This allows
columns `tgt[:,1:end-1]` to be used as the decoder input and `tgt[:,2:end]` as the desired
decoder output.

Assume any shorter sentences in the batches have been padded with extra `eos` tokens on
the left for `src` and on the right for `tgt`. Don't worry about masking `src` for the
encoder, it doesn't have a significant effect on the loss. However do mask `tgt` before
`nll`: you do not want the padding tokens to be counted in the loss calculation.

Please review `@doc RNN`: in particular the `r.c` and `r.h` fields can be used to get/set
the cell and hidden arrays of an RNN (note that `0` and `nothing` act as special values).

RNNs take a dropout value at construction and apply dropout to the input of every layer if
it is non-zero. You need to handle dropout for other layers in the loss function or in
layer definitions as necessary.

In [14]:
function (s::S2S_v1)(src, tgt; average=true) # B, Tx  # B, Ty
    s.encoder.h = 0
    s.encoder.c = 0
    source_embedding = s.srcembed(src)
    s.encoder(source_embedding)
    s.decoder.h = s.encoder.h
    s.decoder.c = s.encoder.c
    dec_in_embedding = s.tgtembed(tgt[:,1:end-1])
    dec_hidden = s.decoder(dec_in_embedding) # H, B, T
    hidden_reshaped = reshape(dec_hidden, (size(dec_hidden,1), size(dec_hidden,2)*size(dec_hidden,3))) # H, B*T
    scores = s.projection(hidden_reshaped) # V, B*T
    target = tgt[:, 2:end] # B, T
    mask!(target, s.tgtvocab.eos)
    target = reshape(target, :)
    nll(scores, target, average=average)
end

In [15]:
@info "Testing S2S_v1"
Knet.seed!(1)
model = S2S_v1(512, 512, 512, tr_vocab, en_vocab; layers=2, bidirectional=true, dropout=0.2)
(x,y) = first(dtst)
# Your loss can be slightly different due to different ordering of words in the vocabulary.
# The reference vocabulary starts with eos, unk, followed by words in decreasing frequency.
@test model(x,y; average=false) == (14097.471f0, 1432)

┌ Info: Testing S2S_v1
└ @ Main In[15]:1


Test Passed

### Loss for a whole dataset

Define a `loss(model, data)` which returns a `(Σloss, Nloss)` pair if `average=false` and
a `Σloss/Nloss` average if `average=true` for a whole dataset. Assume that `data` is an
iterator of `(x,y)` pairs such as `MTData` and `model(x,y;average)` is a model like
`S2S_v1` that computes loss on a single `(x,y)` pair.

In [16]:
function loss(model, data; average=true)
    loss = 0
    count = 0
    Σloss = 0
    Nloss = 0
    for (x, y) in data
        loss, count = model(x, y, average=false)
        Σloss += loss
        Nloss += count
    end
    
    if average
        return Σloss/Nloss
    else
        return Σloss, Nloss
    end
end

loss (generic function with 1 method)

In [17]:
@info "Testing loss"
@test loss(model, dtst, average=false) == (1.0429117f6, 105937)
# Your loss can be slightly different due to different ordering of words in the vocabulary.
# The reference vocabulary starts with eos, unk, followed by words in decreasing frequency.
# Also, because we do not mask src, different batch sizes may lead to slightly different
# losses. The test above gives (1.0429178f6, 105937) with batchsize==1.

┌ Info: Testing loss
└ @ Main In[17]:1


Test Passed

### Training SGD_v1

The following function can be used to train our model. `trn` is the training data, `dev`
is used to determine the best model, `tst...` can be zero or more small test datasets for
loss reporting. It returns the model that does best on `dev`.

In [18]:
function train!(model, trn, dev, tst...)
    bestmodel, bestloss = deepcopy(model), loss(model, dev)
    progress!(adam(model, trn), steps=100) do y
        losses = [ loss(model, d) for d in (dev,tst...) ]
        if losses[1] < bestloss
            bestmodel, bestloss = deepcopy(model), losses[1]
        end
        return (losses...,)
    end
    return bestmodel
end

train! (generic function with 1 method)

You should be able to get under 3.40 dev loss with the following settings in 10
epochs. The training speed on a V100 is about 3 mins/epoch or 40K words/sec, K80 is about
6 times slower. Using settings closer to the Luong paper (per-sentence loss rather than
per-word loss, SGD with lr=1, gclip=1 instead of Adam), you can get to 3.17 dev loss in
about 25 epochs. Using dropout and shuffling batches before each epoch significantly
improve the dev loss. You can play around with hyperparameters but I doubt results will
get much better without attention. To verify your training, here is the dev loss I
observed at the beginning of each epoch in one training session:
`[9.83, 4.60, 3.98, 3.69, 3.52, 3.41, 3.35, 3.32, 3.30, 3.31, 3.33]`

In [24]:
@info "Training S2S_v1"
epochs = 10
ctrn = collect(dtrn)
trnx10 = collect(flatten(shuffle!(ctrn) for i in 1:epochs))
trn20 = ctrn[1:20]
dev38 = collect(ddev)
# Uncomment this to train the model (This takes about 30 mins on a V100):
model = train!(model, trnx10, dev38, trn20)
# Uncomment this to save the model:
# Knet.save("s2s_v1.jld2","model",model)
# Uncomment this to load the model:
# model = Knet.load("s2s_v1.jld2","model")

┌ Info: Training S2S_v1
└ @ Main In[24]:1
┣███████████████████▋┫ [98.39%, 14100/14330, 02:57:30/03:00:23, 1.36i/s] (3.4897823f0, 2.4676006f0)              ┫ [1.40%, 200/14330, 02:43/03:14:16, 1.40i/s] (6.0929995f0, 6.1956534f0)█▊                  ┫ [9.07%, 1300/14330, 16:22/03:00:23, 1.40i/s] (4.6160693f0, 4.637247f0)██▌                 ┫ [12.56%, 1800/14330, 22:38/03:00:09, 1.32i/s] (4.3524394f0, 4.338263f0)██▊                 ┫ [13.96%, 2000/14330, 25:03/02:59:26, 1.45i/s] (4.2677917f0, 4.2499704f0)███▏                ┫ [16.05%, 2300/14330, 28:48/02:59:24, 1.41i/s] (4.1583853f0, 4.144301f0)████▌               ┫ [23.03%, 3300/14330, 41:12/02:58:56, 1.39i/s] (3.9148686f0, 3.8874183f0)█████▏              ┫ [25.82%, 3700/14330, 46:18/02:59:19, 1.27i/s] (3.8186646f0, 3.689507f0)█████▍              ┫ [27.22%, 3900/14330, 48:54/02:59:41, 1.25i/s] (3.7730432f0, 3.6619437f0)█████▋              ┫ [28.61%, 4100/14330, 51:24/02:59:39, 1.32i/s] (3.7492828f0, 3.6441243f0)█████▊              ┫ [29.

### Generating translations

With a single argument, a `S2S_v1` object should take it as a batch of source sentences
and generate translations for them. After passing `src` through the encoder and copying
its hidden states to the decoder, the decoder is run starting with an initial input of all
`eos` tokens. Highest scoring tokens are appended to the output and used as input for the
subsequent decoder steps.  The decoder should stop generating when all sequences in the
batch have generated `eos` or when `stopfactor * size(src,2)` decoder steps are reached. A
correctly shaped target language batch should be returned.

In [25]:
function (s::S2S_v1)(src::Matrix{Int}; stopfactor = 3)
    B = size(src, 1) # batch size
    src_len = size(src, 2) # source maximum sentence length
    eos_ind = s.tgtvocab.eos # eos token (we need it for the stopping condition)
    # start of the processing part
    # reset encoder hidden state at the start of the sentence
    s.encoder.h = 0
    s.encoder.c = 0
    # get the embedding of the source sentence batch
    source_embedding = s.srcembed(src) # dims: E, B, Tx
    # encode the source sentence
    s.encoder(source_embedding)
    # transfer the encoding from the encoder to the decoder
    s.decoder.h = s.encoder.h
    s.decoder.c = s.encoder.c
    # make the first decoder input (batch of eos tokens) # dims: B
    first_word = repeat([eos_ind], B)
    # encode the eos batch # dims: E, B
    emb = s.tgtembed(first_word)
    # keep the generated words in a variable (tgt) # dims: B, Ty
    tgt = []
    # stopping condition:
    # 1- size(tgt, 2) >= stopfactor * src_len
    # 2- if all batches have generate eos (make variable to check that) # dims: B
    eos_generated = zeros(B) # zero if hasn't generated eos, one if eos has been generated
    
    while true # for each decoder timestep
        # feed it to the decoder and get the output # dims: H, B
        dec_out = s.decoder(emb)
        # feed it to the projection layer and get scores over vocab # dims: V, B
        out_score = s.projection(dec_out)
        # get the index of the maximum score for each batch element # dims: B
        predicted_tokens = reshape(map(x -> x[1], argmax(out_score, dims=1)), :)
        eoses = findall(x->x==eos_ind, predicted_tokens)
        eos_generated[eoses] .= 1
        push!(tgt, predicted_tokens)
        if (length(tgt) >= stopfactor * src_len) || all(eos_generated .== 1)
            break
        end
        # encode the indices from the prev step # dims: E, B
        emb = s.tgtembed(predicted_tokens)
    end
    tgt = hcat(tgt...)
    return tgt # dims: B, Ty
end

In [26]:
# Utility to convert int arrays to sentence strings
function int2str(y,vocab)
    y = vec(y)
    ysos = findnext(w->!isequal(w,vocab.eos), y, 1)
    ysos == nothing && return ""
    yeos = something(findnext(isequal(vocab.eos), y, ysos), 1+length(y))
    join(vocab.i2w[y[ysos:yeos-1]], " ")
end

int2str (generic function with 1 method)

In [27]:
@info "Generating some translations"
d = MTData(tr_dev, en_dev, batchsize=1) |> collect
(src,tgt) = rand(d)
out = model(src)
println("SRC: ", int2str(src,model.srcvocab))
println("REF: ", int2str(tgt,model.tgtvocab))
println("OUT: ", int2str(out,model.tgtvocab))
# Here is a sample output:
# SRC: çin'e 15 şubat 2006'da ulaştım .
# REF: i made it to china on february 15 , 2006 .
# OUT: i got to china , china , at the last 15 years .

┌ Info: Generating some translations
└ @ Main In[27]:1


SRC: ( kahkahalar ) 17 yıl boyunca sessiz <unk> hayal edin , ve anneniz seyirciler arasından , böyle der .
REF: ( laughter ) imagine if you were quiet for 17 years and your mother was out in the audience , say .
OUT: ( laughter ) if you go back to the <unk> , you 'll see that the <unk> of this year , and this is the <unk> .


### Calculating BLEU

BLEU is the most commonly used metric to measure translation quality. The following should
take a model and some data, generate translations and calculate BLEU.

In [28]:
function bleu(s2s,d::MTData)
    d = MTData(d.src,d.tgt,batchsize=1)
    reffile = d.tgt.file
    hypfile,hyp = mktemp()
    for (x,y) in progress(collect(d))
        g = s2s(x)
        for i in 1:size(y,1)
            println(hyp, int2str(g[i,:], d.tgt.vocab))
        end
    end
    close(hyp)
    isfile("multi-bleu.perl") || download("https://github.com/moses-smt/mosesdecoder/raw/master/scripts/generic/multi-bleu.perl", "multi-bleu.perl")
    run(pipeline(`cat $hypfile`,`perl multi-bleu.perl $reffile`))
    return hypfile
end

bleu (generic function with 1 method)

Calculating dev BLEU takes about 45 secs on a V100. We get about 8.0 BLEU which is pretty
low. As can be seen from the sample translations a loss of ~3+ (perplexity ~20+) or a BLEU
of ~8 is not sufficient to generate meaningful translations.

In [29]:
@info "Calculating BLEU"
bleu(model, ddev)

┌ Info: Calculating BLEU
└ @ Main In[29]:1
┣████████████████████┫ [100.00%, 4045/4045, 01:40/01:40, 40.55i/s] 
perl: warning: Setting locale failed.
perl: warning: Please check that your locale settings:
	LANGUAGE = (unset),
	LC_ALL = (unset),
	LC_CTYPE = "UTF-8",
	LANG = "en_US.UTF-8"
    are supported and installed on your system.
perl: warning: Falling back to the standard locale ("C").


BLEU = 7.81, 36.8/11.4/4.6/1.9 (BP=1.000, ratio=1.079, hyp_len=88979, ref_len=82502)


It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl can still be used for internal purposes when you have a consistent tokenizer.


"/tmp/jl_vdyBFF"

To improve the quality of translations we can use more training data, different training
and model parameters, or preprocess the input/output: e.g. splitting Turkish words to make
suffixes look more like English function words may help. Other architectures,
e.g. attention and transformer, perform significantly better than this simple S2S model.

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*